In [11]:
# import for saving data in google form
import requests
from bs4 import BeautifulSoup
import json
import time

# mport for saving data in disk by using panda
import pandas as pd
import os

# import for using button 
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

#import for running quiz
from IPython.display import Image

In [12]:
# Apply button
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=10)
    text_input.disabled = True
    return event['description']

In [13]:
# Consent about collecting data
def get_data_consent():
    data_consent_info = """DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type   yes   in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ").lower() 
    if result == "yes": 
        print("Thanks for your participation.")
        print("Please contact philip.lewis@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
    else: 
        raise Exception("User did not consent to continue test.")

In [18]:
# Test Introduction
def get_user_info():
    print("Welcome to the spatial reasoning test")
    print("Please identify a 2D projection that cannot be made by rotating this arrangement in space. There are 10 questions in total!")
    time.sleep(6)
    clear_output(wait=False)

    username = ""
    while True:
        id_instructions = """
        Enter your anonymized ID
        To generate an anonymous 4-letter unique user identifier, please enter:
        - two letters based on the initials (first and last name) of a childhood friend
        - two letters based on the initials (first and last name) of a favorite actor / actress
        e.g. if your friend was called Charlie Brown and film star was Tom Cruise
        then your unique identifier would be CBTC
        """
        print(id_instructions)
        username = input("> ").upper()
        if len(username) == 4 and username.isalpha():
            clear_output(wait=False)
            break
        else:
            print("Please enter a 4-letter ID containing only letters. Try again.")
            clear_output(wait=False)

    while True:
        print('Are you male or female? (Enter "male" or "female")')
        username_gender = input(">").lower()
        if username_gender in ['male', 'female']:
            clear_output(wait=False)
            break
        else:
            print("Invalid input. Please enter 'male' or 'female'. Try again.")
            clear_output(wait=False)

    print('How old are you?')
    username_age = input("> ")
    clear_output(wait=False)

    print('Have you drunk any alcoholic beverages in the last 24h? Please answer yes or no')
    username_answer = input('>').lower()
    clear_output(wait=False)

    print("Let's start the quiz", username)
    time.sleep(2)
    clear_output(wait=False)

    return username, username_age, username_gender, username_answer

In [19]:
# Full quiz
def run_quiz(username, picture_list, answer_list, event_info):
    start_time = time.time()
    time_limit = 180
    answer_each = []
    time_each_question = []
    question_data = []

    for i in range(len(answer_list)):
        question_start_time = time.time()
        btn1 = widgets.Button(description="a")
        btn2 = widgets.Button(description="b")
        btn3 = widgets.Button(description="c")
        btn4 = widgets.Button(description="d")

        btn1.on_click(register_btn_event)
        btn2.on_click(register_btn_event)
        btn3.on_click(register_btn_event)
        btn4.on_click(register_btn_event)

        myhtml1 = HTML("<h3>Which of the views (a-d) can not be made by rotating the cube arrangement shown?<h3>")
        display(myhtml1)
        myhtml2 = HTML("<h4>Please answer as accurately as possible </h4>")
        display(myhtml2)

        display(picture_list[i])

        panel = widgets.HBox([btn1, btn2, btn3, btn4])
        display(panel)
        result = wait_for_event(interval=1)
        clear_output()

        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time up. Quiz terminated.")
            break

        time_taken_for_question = time.time() - question_start_time
        time_each_question.append(time_taken_for_question)

        if result['description'] == answer_list[i]:
            print('Well done!')
            time.sleep(2)
            score_each = 1
        else:
            print('Wrong!')
            print('Have a cup of coffee to wake!')
            time.sleep(2)
            score_each = 0

        answer_each.append(score_each)
        clear_output(wait=False)

        # Save question data
        question_data.append({
            'Question': f"Question {i + 1}",
            'User Answer': result['description'],
            'Score': 'Correct' if score_each == 1 else 'Wrong' if score_each == 0 else 'Not Answered Yet',
            'Correct Answer': answer_list[i],
            'Time Taken': time_taken_for_question
        })

    end_time = time.time()
    time_taken = end_time - start_time

    # Create a DataFrame with question data
    df_question_data = pd.DataFrame(question_data)

    # Calculate accuracy
    num_answered = len(answer_each)  # Total number of questions attempted
    num_correct = sum(answer_each)   # Total number of correct answers
    if num_answered > 0:
        accuracy = (num_correct / num_answered) * 100  
    else:
        accuracy = 0  # In case no questions were answered

    print("Quiz over", username)
    print(f"You got {num_correct} correct out of {num_answered} questions attempted.")
    print(f"This gives you an accuracy of {accuracy:.2f}%.")
    print(f"You have used in total {time_taken:.2f} seconds to complete your test.")

    # Display question data DataFrame
    print("\nQuestion-wise Data:")
    display(df_question_data)

    return username, accuracy, time_taken, answer_each, df_question_data

In [20]:
# def send_to_google_formUpload data to google form
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [21]:
# Run the quiz

#Call consent and collecting infomation
get_data_consent()   
username, username_age, username_gender, username_answer = get_user_info()

# Run function run_quiz
picture_list = [
    Image('cube1.png', width=500),
    Image('cube2.png', width=500),
    Image('cube8.png', width=500),
    Image('cube7.png', width=500),
    Image('cube5.png', width=500),
    Image('cube9.png', width=500),
    Image('cube10.png', width=500),
    Image('cube6.png', width=500),
    Image('cube4.png', width=500),
    Image('cube3.png', width=500),
    ]
answer_list = ['d', 'b', 'b', 'd', 'd', 'd', 'a', 'c', 'b', 'd']
event_info = {
    'type': '',
    'description': '',
    'time': -1
}
username, accuracy, time_taken, answer_each, df_question_data = run_quiz(username, picture_list, answer_list, event_info)
    

# Run function send_to_google_form
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSey3_gng-x5AonNld3n1spZdtdHIMMSiXfYjJr2dF7YwYJJUw/viewform?usp=sf_link"
data_dict = {'name': username,
              'age': username_age,
              'gender': username_gender,
              'alcoholic beverage': username_answer,
              'grade': accuracy,
              'time': time_taken,
              }
for i in range(1, 11):
    key = f'answer_{i}'
    if i <= len(answer_each):
        data_dict[key] = answer_each[i - 1]  
    else:
        data_dict[key] = 'Not Answered'

send_to_google_form(data_dict, form_url)

Time up. Quiz terminated.
Quiz over BBBB
You got 1 correct out of 4 questions attempted.
This gives you an accuracy of 25.00%.
You have used in total 241.89 seconds to complete your test.

Question-wise Data:


,Question,User Answer,Score,Correct Answer,Time Taken
0,Question 1,b,Wrong,d,6.066425
1,Question 2,c,Wrong,b,4.092829
2,Question 3,b,Correct,b,2.092604
3,Question 4,b,Wrong,d,3.026776


True